# Примеры запросов

## Параметры пути

### Простой запрос без параметров

In [65]:
! http GET localhost:5002

25520.35s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 200 OK
content-length: 19
content-type: application/json
date: Mon, 02 Sep 2024 16:55:13 GMT
server: uvicorn

{
    "message": "Hello"
}




### Получить все значения

In [2]:
! http GET localhost:5002/items

HTTP/1.1 200 OK
content-length: 31
content-type: application/json
date: Mon, 02 Sep 2024 09:52:03 GMT
server: uvicorn

{
    "message": [
        "item 1",
        "item 2"
    ]
}




### Не верный путь - ошибка 404 

In [3]:
! http GET localhost:5002/ite

HTTP/1.1 404 Not Found
content-length: 22
content-type: application/json
date: Mon, 02 Sep 2024 09:52:20 GMT
server: uvicorn

{
    "detail": "Not Found"
}




### Получить сущность по id

In [4]:
! http GET localhost:5002/items/2

HTTP/1.1 200 OK
content-length: 13
content-type: application/json
date: Mon, 02 Sep 2024 09:53:36 GMT
server: uvicorn

{
    "item_id": 2
}




### Проверка валидности параметров пути

In [6]:
# Неправильный параметр
! http GET localhost:5002/items_validated/4

HTTP/1.1 422 Unprocessable Entity
content-length: 211
content-type: application/json
date: Mon, 02 Sep 2024 09:55:03 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "ge": 10
            },
            "input": "4",
            "loc": [
                "path",
                "request_id"
            ],
            "msg": "Input should be greater than or equal to 10",
            "type": "greater_than_equal",
            "url": "https://errors.pydantic.dev/2.4/v/greater_than_equal"
        }
    ]
}




In [7]:
# Правильный параметр
! http GET localhost:5002/items_validated/12

HTTP/1.1 200 OK
content-length: 17
content-type: application/json
date: Mon, 02 Sep 2024 09:55:15 GMT
server: uvicorn

{
    "request_id": 12
}




### Использование перечислений

In [8]:
# Правильный параметр
! http GET localhost:5002/requests/type/service

HTTP/1.1 200 OK
content-length: 26
content-type: application/json
date: Mon, 02 Sep 2024 09:56:03 GMT
server: uvicorn

{
    "request_type": "service"
}




In [9]:
# Неправильный параметр
! http GET localhost:5002/requests/type/services

HTTP/1.1 422 Unprocessable Entity
content-length: 176
content-type: application/json
date: Mon, 02 Sep 2024 09:56:30 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "expected": "'service' or 'new_feature'"
            },
            "input": "services",
            "loc": [
                "path",
                "request_type"
            ],
            "msg": "Input should be 'service' or 'new_feature'",
            "type": "enum"
        }
    ]
}




## Параметры строки запроса

### Проверка на соотвествие типа

In [13]:
# строка
! http GET 'localhost:5002/test?text=test string'

HTTP/1.1 200 OK
content-length: 30
content-type: application/json
date: Mon, 02 Sep 2024 09:58:37 GMT
server: uvicorn

{
    "test_request": "test string"
}




In [14]:
# число
! http GET 'localhost:5002/num_test?num=12'

HTTP/1.1 200 OK
content-length: 10
content-type: application/json
date: Mon, 02 Sep 2024 09:59:04 GMT
server: uvicorn

{
    "num": 12
}




In [15]:
# Неправильный параметр
! http GET 'localhost:5002/num_test?text=12'

HTTP/1.1 422 Unprocessable Entity
content-length: 139
content-type: application/json
date: Mon, 02 Sep 2024 09:59:29 GMT
server: uvicorn

{
    "detail": [
        {
            "input": null,
            "loc": [
                "query",
                "num"
            ],
            "msg": "Field required",
            "type": "missing",
            "url": "https://errors.pydantic.dev/2.4/v/missing"
        }
    ]
}




In [16]:
# Один обязательный, остальные задаы по умолчанию
! http GET 'localhost:5002/query_params?req=test string'

HTTP/1.1 200 OK
content-length: 39
content-type: application/json
date: Mon, 02 Sep 2024 10:00:45 GMT
server: uvicorn

{
    "page": 0,
    "req": "test string",
    "skip": 0
}




In [18]:
# Проверка на соотвествие условиям
! http GET 'localhost:5002/query_params_typed?req=test'

HTTP/1.1 422 Unprocessable Entity
content-length: 208
content-type: application/json
date: Mon, 02 Sep 2024 10:02:27 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "min_length": 5
            },
            "input": "test",
            "loc": [
                "query",
                "req"
            ],
            "msg": "String should have at least 5 characters",
            "type": "string_too_short",
            "url": "https://errors.pydantic.dev/2.4/v/string_too_short"
        }
    ]
}




In [26]:
# Проверка на соотвествие условиям со значением по умолчанию
! http GET 'localhost:5002/query_params_default'

HTTP/1.1 200 OK
content-length: 25
content-type: application/json
date: Mon, 02 Sep 2024 10:04:52 GMT
server: uvicorn

{
    "req": "default striing"
}




In [30]:
# Проверка на соотвествие условиям по регулярному выражению
! http GET 'localhost:5002/query_params_regular?req=testd'

HTTP/1.1 200 OK
content-length: 15
content-type: application/json
date: Mon, 02 Sep 2024 10:06:35 GMT
server: uvicorn

{
    "req": "testd"
}




In [33]:
# Получение списка условий
! http GET 'localhost:5002/query_params_list?q=testd&q=foo'

HTTP/1.1 200 OK
content-length: 21
content-type: application/json
date: Mon, 02 Sep 2024 10:08:25 GMT
server: uvicorn

{
    "q": [
        "testd",
        "foo"
    ]
}




In [37]:
# Cписок условий со значениями по умолчанию
! http GET 'localhost:5002/query_params_list_default'

HTTP/1.1 200 OK
content-length: 19
content-type: application/json
date: Mon, 02 Sep 2024 10:09:49 GMT
server: uvicorn

{
    "q": [
        "foo",
        "bar"
    ]
}




## Параметры тела запроса

In [46]:
! http POST localhost:5002 \
    name=user

HTTP/1.1 200 OK
content-length: 261
content-type: application/json
date: Mon, 02 Sep 2024 10:17:24 GMT
server: uvicorn

{
    "body": "{\"name\": \"user\"}",
    "cooke": {},
    "headers": {
        "accept": "application/json, */*;q=0.5",
        "accept-encoding": "gzip, deflate",
        "connection": "keep-alive",
        "content-length": "16",
        "content-type": "application/json",
        "host": "localhost:5002",
        "user-agent": "HTTPie/3.2.2"
    }
}




In [47]:
! http --form PUT localhost:5002/items \
username='user q' \
password='pass'

HTTP/1.1 200 OK
content-length: 39
content-type: application/json
date: Mon, 02 Sep 2024 10:19:19 GMT
server: uvicorn

{
    "password": "pass",
    "username": "user q"
}




# Группировка зависимостей

In [55]:
# часто используемые параметры в отдельном методе
! http localhost:5001/items/

2587.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 200 OK
content-length: 31
content-type: application/json
date: Mon, 02 Sep 2024 10:32:59 GMT
server: uvicorn

{
    "limit": 100,
    "q": null,
    "skip": 0
}




In [57]:
# Зависимости, сгруппированные в класс
! http localhost:5001/class_items

2792.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 200 OK
content-length: 71
content-type: application/json
date: Mon, 02 Sep 2024 10:36:25 GMT
server: uvicorn

{
    "items": [
        {
            "item_name": "Foo"
        },
        {
            "item_name": "Bar"
        },
        {
            "item_name": "Baz"
        }
    ]
}




In [58]:
# Вложенные зависимости
! http localhost:5001/cookie_items

2929.73s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 200 OK
content-length: 20
content-type: application/json
date: Mon, 02 Sep 2024 10:38:42 GMT
server: uvicorn

{
    "q_or_cookie": null
}




In [59]:
# Вложенные зависимости - принять параметр
! http 'localhost:5001/cookie_items?q=test str'

2971.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 200 OK
content-length: 26
content-type: application/json
date: Mon, 02 Sep 2024 10:39:23 GMT
server: uvicorn

{
    "q_or_cookie": "test str"
}




In [63]:
# Валидация параметров
! http localhost:5001/validate_items \
    'x-token: fake-super-secret-token' \
    'x-key: fake-super-secret-key'

3389.89s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 200 OK
content-length: 31
content-type: application/json
date: Mon, 02 Sep 2024 10:46:23 GMT
server: uvicorn

[
    {
        "item": "Foo"
    },
    {
        "item": "Bar"
    }
]




In [64]:
# Валидация параметров
! http localhost:5001/validate_items

3415.55s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


HTTP/1.1 422 Unprocessable Entity
content-length: 274
content-type: application/json
date: Mon, 02 Sep 2024 10:46:48 GMT
server: uvicorn

{
    "detail": [
        {
            "input": null,
            "loc": [
                "header",
                "x-token"
            ],
            "msg": "Field required",
            "type": "missing",
            "url": "https://errors.pydantic.dev/2.4/v/missing"
        },
        {
            "input": null,
            "loc": [
                "header",
                "x-key"
            ],
            "msg": "Field required",
            "type": "missing",
            "url": "https://errors.pydantic.dev/2.4/v/missing"
        }
    ]
}


